In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set, desc

In [2]:
spark = SparkSession.builder.appName('main').getOrCreate()

22/09/01 11:48:46 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlp4s0)
22/09/01 11:48:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/01 11:48:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataset_path = 'inputs/by-company.csv'
output_path = 'outputs/by-company'

In [4]:
def check(df):
    print(f"Count: {df.count()}")
    print('\nElements:')
    print(df.show(5, truncate = False))

In [5]:
df = spark.read.options(header = True).csv(dataset_path)

In [6]:
# check(df)

In [7]:
df = df.groupBy('sale_code').agg(collect_set('barcode'))

In [8]:
df = df.withColumnRenamed('collect_set(barcode)', 'items')


In [9]:
# check(df)

In [10]:
fp_growth = FPGrowth(itemsCol = 'items', minSupport = 0.0005, minConfidence = 0.001)

In [11]:
model = fp_growth.fit(df)

In [12]:
freq_itemsets = model.freqItemsets

In [13]:
freq_itemsets = freq_itemsets.select('freq', 'items')

In [14]:
freq_itemsets = freq_itemsets.sort(col('freq').desc())

In [15]:
# check(freq_itemsets)

In [16]:
rules = model.associationRules

In [17]:
rules = rules.sort(col('confidence').desc(), col('lift').desc())

In [18]:
# check(rules)

In [19]:
transformed = model.transform(df)

In [20]:
# check(transformed)

In [21]:
freq_itemsets.toPandas().to_csv(f"{output_path}/frequent_itemsets.csv", index = False)

In [22]:
rules.toPandas().to_csv(f"{output_path}/association_rules.csv", index = False)

In [23]:
transformed.toPandas().to_csv(f"{output_path}/transformed.csv", index = False)